## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Vaini,TO,2021-06-03 19:39:36,-21.2000,-175.2000,68.00,94,40,8.95,scattered clouds
1,1,Butaritari,KI,2021-06-03 19:39:36,3.0707,172.7902,82.85,75,100,10.09,overcast clouds
2,2,Arraial Do Cabo,BR,2021-06-03 19:39:36,-22.9661,-42.0278,82.36,61,0,5.75,clear sky
3,3,Tibiao,PH,2021-06-03 19:39:37,11.2915,122.0354,80.78,82,100,5.64,overcast clouds
4,4,Castro,BR,2021-06-03 19:39:37,-24.7911,-50.0119,65.93,81,12,2.46,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input('Enter the maximum temperature for your ideal vacation destination'))
min_temp = float(input('Enter the minimum temperature for your ideal vacation destination'))

Enter the maximum temperature for your ideal vacation destination85
Enter the minimum temperature for your ideal vacation destination65


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
ideal_destinations = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
ideal_destinations

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Vaini,TO,2021-06-03 19:39:36,-21.2000,-175.2000,68.00,94,40,8.95,scattered clouds
1,1,Butaritari,KI,2021-06-03 19:39:36,3.0707,172.7902,82.85,75,100,10.09,overcast clouds
2,2,Arraial Do Cabo,BR,2021-06-03 19:39:36,-22.9661,-42.0278,82.36,61,0,5.75,clear sky
3,3,Tibiao,PH,2021-06-03 19:39:37,11.2915,122.0354,80.78,82,100,5.64,overcast clouds
4,4,Castro,BR,2021-06-03 19:39:37,-24.7911,-50.0119,65.93,81,12,2.46,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
670,670,Den Helder,NL,2021-06-03 19:43:51,52.9599,4.7593,65.98,88,90,7.00,mist
672,672,Mlimba,TZ,2021-06-03 19:43:51,-8.8000,35.8167,69.37,93,92,2.80,light rain
673,673,San Luis,AR,2021-06-03 19:43:52,-33.2950,-66.3356,70.00,42,42,14.20,scattered clouds
676,676,Popondetta,PG,2021-06-03 19:43:52,-8.7537,148.2534,74.01,95,100,2.48,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
ideal_destinations.count()

City_ID                331
City                   331
Country                328
Date                   331
Lat                    331
Lng                    331
Max Temp               331
Humidity               331
Cloudiness             331
Wind Speed             331
Current description    331
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = ideal_destinations.dropna()
clean_df.count()

City_ID                328
City                   328
Country                328
Date                   328
Lat                    328
Lng                    328
Max Temp               328
Humidity               328
Cloudiness             328
Wind Speed             328
Current description    328
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Vaini,TO,68.00,scattered clouds,-21.2000,-175.2000,
1,Butaritari,KI,82.85,overcast clouds,3.0707,172.7902,
2,Arraial Do Cabo,BR,82.36,clear sky,-22.9661,-42.0278,
3,Tibiao,PH,80.78,overcast clouds,11.2915,122.0354,
4,Castro,BR,65.93,few clouds,-24.7911,-50.0119,
12,Guarapari,BR,80.49,clear sky,-20.6667,-40.4975,
15,Florianopolis,BR,67.26,broken clouds,-27.6146,-48.5012,
16,Hithadhoo,MV,83.66,overcast clouds,-0.6000,73.0833,
17,Rikitea,PF,73.11,overcast clouds,-23.1203,-134.9692,
18,Torbay,CA,73.40,broken clouds,47.6666,-52.7314,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna().count()
hotel_df

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Vaini,TO,68.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Butaritari,KI,82.85,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
2,Arraial Do Cabo,BR,82.36,clear sky,-22.9661,-42.0278,Pousada Porto Praia
3,Tibiao,PH,80.78,overcast clouds,11.2915,122.0354,Kasa Raya Inn
4,Castro,BR,65.93,few clouds,-24.7911,-50.0119,CHACARA BAILLY
...,...,...,...,...,...,...,...
670,Den Helder,NL,65.98,mist,52.9599,4.7593,Hotel Den Helder
672,Mlimba,TZ,69.37,light rain,-8.8000,35.8167,California Lodge
673,San Luis,AR,70.00,scattered clouds,-33.2950,-66.3356,Huarpes Hotel
676,Popondetta,PG,74.01,overcast clouds,-8.7537,148.2534,Timos Transit House


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current description</dt><dd>{Current description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))